In [1]:
# =============================================================================
# MODEL 4: Hibrit CNN Özellik Çıkarıcı + Geleneksel ML Sınıflandırıcı
# Bu notebook, bir CNN'in özellik çıkarıcı kısmını kullanarak görüntü özelliklerini çıkarır,
# bu özellikleri ve etiketleri .npy dosyalarına kaydeder, ardından bu verilerle
# geleneksel bir makine öğrenmesi modelini eğitir ve test eder.
# =============================================================================

# =============================================================================
# 1. Gerekli Kütüphaneleri İçe Aktarma
# =============================================================================
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data import DataLoader

import numpy as np
import os # Dosya işlemleri için

from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

print("Gerekli kütüphaneler içe aktarıldı.")

# =============================================================================
# 2. Cihazı Ayarlama (GPU veya CPU)
# =============================================================================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Kullanılan cihaz: {device}")

# =============================================================================
# 3. Veri Dönüşümleri ve Yükleyiciler
# CNN özellik çıkarıcısı için ImageNet boyutları ve normalizasyonu kullanılır.
# =============================================================================
# ImageNet standardı için dönüşümler
transform = transforms.Compose([
    transforms.Resize((224, 224)), # ResNet/VGG gibi modeller 224x224 bekler
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # ImageNet normalizasyonu
])

# CIFAR-10 Veri Seti
# download=True ilk çalıştırmada veriyi indirir
train_data = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_data = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Veri Yükleyiciler
# num_workers > 0 kullanmak veri yüklemeyi hızlandırabilir, ancak Jupyter'de dikkatli kullanılmalı
# Jupyter'da sorun yaşarsanız num_workers=0 yapın
train_loader = DataLoader(train_data, batch_size=64, shuffle=False, num_workers=2) # Shuffle=False özellik çıkarma sırasını korur
test_loader = DataLoader(test_data, batch_size=64, shuffle=False, num_workers=2)

print(f"Eğitim veri seti boyutu: {len(train_data)}")
print(f"Test veri seti boyutu: {len(test_data)}")

# CIFAR-10 sınıf isimleri (raporlama için)
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck']


# =============================================================================
# 4. CNN Özellik Çıkarıcıyı Hazırlama
# ImageNet üzerinde önceden eğitilmiş bir ResNet-18 kullanacağız.
# Son sınıflandırma katmanını çıkaracağız.
# =============================================================================
# ImageNet üzerinde önceden eğitilmiş ResNet-18 modelini yükle
# pretrained=True, ImageNet ağırlıklarını yükler
cnn_model = models.resnet18(pretrained=True)

# Özellik çıkarıcıyı oluşturma: ResNet'in 'fc' (fully connected) katmanını çıkar
# Sequential(*list(cnn_model.children())[:-1]) ifadesi, modelin tüm alt katmanlarını alıp
# sonuncusunu (yani 'fc'yi) listeden çıkarır ve bunlardan yeni bir Sequential modeli oluşturur.
feature_extractor = torch.nn.Sequential(*list(cnn_model.children())[:-1])

# Özellik çıkarıcıyı çıkarım moduna al
feature_extractor.eval()

# Özellik çıkarıcıyı seçilen cihaza taşı
feature_extractor = feature_extractor.to(device)

print("\nCNN özellik çıkarıcı (ResNet-18, son katman hariç) hazırlandı.")
print(f"Özellik çıkarıcı modeli kullanılan cihaz: {next(feature_extractor.parameters()).device}")

# =============================================================================
# 5. Veri Setinden Özellikleri ve Etiketleri Çıkarma
# Bu adım uzun sürebilir. Çıkarılan veriler .npy olarak kaydedilecektir.
# =============================================================================

# Gradyan hesaplamayı kapat (çıkarım yaptığımız için gerek yok, bellekte tasarruf sağlar)
torch.set_grad_enabled(False)

def extract_features(loader, model_fe, device):
    all_features = []
    all_labels = []

    print(f"Özellikler {len(loader.dataset)} görüntü için çıkarılıyor...")

    for i, (images, labels) in enumerate(loader):
        images = images.to(device)

        # Özellik çıkarıcıdan geçir
        features = model_fe(images)

        # Özellik vektörünü düzleştir (flatten)
        # ResNet'in AvgPool sonrası çıktısı (batch_size, num_features, 1, 1) şeklindedir.
        # Bunu (batch_size, num_features) şekline düzleştiriyoruz.
        features = torch.flatten(features, start_dim=1)

        # CPU'ya taşı ve numpy dizisine çevir, listelere ekle
        all_features.append(features.cpu().numpy())
        all_labels.append(labels.cpu().numpy())

        # İlerleme göstergesi (isteğe bağlı)
        if (i + 1) % 50 == 0:
            print(f"  Batch {i+1}/{len(loader)} tamamlandı.")

    # Tüm batch'lerdeki verileri tek bir numpy dizisinde birleştir
    all_features = np.concatenate(all_features, axis=0)
    all_labels = np.concatenate(all_labels, axis=0)

    return all_features, all_labels

# Eğitim ve test veri setleri için özellikleri çıkar
train_features, train_labels = extract_features(train_loader, feature_extractor, device)
test_features, test_labels = extract_features(test_loader, feature_extractor, device)

print("\nÖzellik çıkarma tamamlandı.")
print(f"Eğitim özellikleri şekli: {train_features.shape}")
print(f"Test özellikleri şekli: {test_features.shape}")


# =============================================================================
# 6. Özellikleri ve Etiketleri .npy Dosyalarına Kaydetme
# Bu adım, özellikleri bir kere çıkarıp sonraki çalıştırmalarda tekrar kullanmak için iyidir.
# =============================================================================
# Kaydedilecek dosya yollarını belirle
output_dir = './hybrid_features'
os.makedirs(output_dir, exist_ok=True) # Klasörü oluştur (varsa geç)

train_features_file = os.path.join(output_dir, 'train_features.npy')
train_labels_file = os.path.join(output_dir, 'train_labels.npy')
test_features_file = os.path.join(output_dir, 'test_features.npy')
test_labels_file = os.path.join(output_dir, 'test_labels.npy')

# Numpy formatında kaydet
print(f"\nÖzellikler ve etiketler '{output_dir}' klasörüne kaydediliyor...")
np.save(train_features_file, train_features)
np.save(train_labels_file, train_labels)
np.save(test_features_file, test_features)
np.save(test_labels_file, test_labels)
print("Kaydetme tamamlandı.")


# =============================================================================
# 7. Kaydedilmiş Özellikleri ve Etiketleri Yükleme (İsteğe Bağlı)
# Eğer notebook'u yeniden başlatırsanız, özellik çıkarma adımını atlayıp buradan devam edebilirsiniz.
# =============================================================================
# print(f"\nKayıtlı özellikler ve etiketler '{output_dir}' klasöründen yükleniyor...")
# loaded_train_features = np.load(train_features_file)
# loaded_train_labels = np.load(train_labels_file)
# loaded_test_features = np.load(test_features_file)
# loaded_test_labels = np.load(test_labels_file)
# print("Yükleme tamamlandı.")
#
# # Yüklenen verileri kullanmak için değişkenlere atama
# train_features = loaded_train_features
# train_labels = loaded_train_labels
# test_features = loaded_test_features
# test_labels = loaded_test_labels
# print("Yüklenen veriler atandı.")


# =============================================================================
# 8. Geleneksel Makine Öğrenmesi Modelini Seçme, Eğitme ve Test Etme
# Özellikler üzerinde geleneksel bir sınıflandırıcı eğiteceğiz.
# =============================================================================

# Kullanılacak Geleneksel ML modelini seçin ve başlatın
# Alternatif: Destek Vektör Makinesi (SVC)
model_ml = SVC(kernel='rbf', C=10, gamma='scale', random_state=42) # Ayarları deneme gerektirebilir
print(f"\nSeçilen Geleneksel ML Modeli: {type(model_ml).__name__}")
print("SVC modeli eğitiliyor...")
# SVC eğitimi, Random Forest'a göre daha uzun sürebilir
model_ml.fit(train_features, train_labels)
print("SVC eğitimi tamamlandı.")


# Modeli test veri seti üzerinde değerlendir
print("\nGeleneksel ML modeli test ediliyor...")
predictions = model_ml.predict(test_features)

# Performans metriklerini hesapla
accuracy_ml = accuracy_score(test_labels, predictions)
report_ml = classification_report(test_labels, predictions, target_names=classes) # CIFAR-10 sınıf isimlerini kullan

print(f"\nHibrit Model (CNN Özellikleri + {type(model_ml).__name__}) Test Doğruluğu: {accuracy_ml:.4f}")
print("\nSınıflandırma Raporu:")
print(report_ml)

print("\nModel 4 (Hibrit) süreci tamamlandı.")


Gerekli kütüphaneler içe aktarıldı.
Kullanılan cihaz: cuda
Eğitim veri seti boyutu: 50000
Test veri seti boyutu: 10000


C:\Users\Ege\AppData\Roaming\Python\Python311\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Ege\AppData\Roaming\Python\Python311\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)



CNN özellik çıkarıcı (ResNet-18, son katman hariç) hazırlandı.
Özellik çıkarıcı modeli kullanılan cihaz: cuda:0
Özellikler 50000 görüntü için çıkarılıyor...
  Batch 50/782 tamamlandı.
  Batch 100/782 tamamlandı.
  Batch 150/782 tamamlandı.
  Batch 200/782 tamamlandı.
  Batch 250/782 tamamlandı.
  Batch 300/782 tamamlandı.
  Batch 350/782 tamamlandı.
  Batch 400/782 tamamlandı.
  Batch 450/782 tamamlandı.
  Batch 500/782 tamamlandı.
  Batch 550/782 tamamlandı.
  Batch 600/782 tamamlandı.
  Batch 650/782 tamamlandı.
  Batch 700/782 tamamlandı.
  Batch 750/782 tamamlandı.
Özellikler 10000 görüntü için çıkarılıyor...
  Batch 50/157 tamamlandı.
  Batch 100/157 tamamlandı.
  Batch 150/157 tamamlandı.

Özellik çıkarma tamamlandı.
Eğitim özellikleri şekli: (50000, 512)
Test özellikleri şekli: (10000, 512)

Özellikler ve etiketler './hybrid_features' klasörüne kaydediliyor...
Kaydetme tamamlandı.

Seçilen Geleneksel ML Modeli: SVC
SVC modeli eğitiliyor...
SVC eğitimi tamamlandı.

Geleneksel ML